 https://www.kaggle.com/code/rkuo2000/chromadb-langchain-gpt4all

https://python.langchain.com/docs/integrations/llms/gpt4all

https://github.com/nomic-ai/gpt4all/tree/main/gpt4all-bindings/python

https://www.kaggle.com/code/gpreda/test-gpt4all-on-kaggle

In [1]:
import time
start = time.time()

In [2]:
! nvidia-smi -L

/bin/bash: nvidia-smi: command not found


In [3]:
%%time

! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers
! pip install -qq -U bitsandbytes bitsandbytes-cuda117
! pip install -qq -U gpt4all

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.0.11 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.2 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 29.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.8.1 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requi

In [4]:
#!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin models

In [5]:
import warnings
warnings.filterwarnings("ignore")

import os
import textwrap
import shutil
import pandas as pd
import glob

import langchain
from langchain.llms import HuggingFacePipeline

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

#quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

#os.environ['TRANSFORMERS_CACHE'] = 'cache/'

print(langchain.__version__)

2024-02-03 18:34:57.969316: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 18:34:57.969458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 18:34:58.210853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


0.1.5


In [6]:
### Multi-document retriever
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
#from gpt4all import GPT4All
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [7]:
!wget https://gpt4all.io/models/gguf/replit-code-v1_5-3b-q4_0.gguf

--2024-02-03 18:35:18--  https://gpt4all.io/models/gguf/replit-code-v1_5-3b-q4_0.gguf
Resolving gpt4all.io (gpt4all.io)... 172.67.71.169, 104.26.1.159, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|172.67.71.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1870449696 (1.7G)
Saving to: 'replit-code-v1_5-3b-q4_0.gguf'

replit-code-v1_5-3b 100%[===================>]   1.74G  33.8MB/s    in 50s     

2024-02-03 18:36:08 (35.8 MB/s) - 'replit-code-v1_5-3b-q4_0.gguf' saved [1870449696/1870449696]



In [8]:

llm = GPT4All(model="/kaggle/working/replit-code-v1_5-3b-q4_0.gguf")

OSError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /opt/conda/lib/python3.10/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllmodel.so)

In [ ]:
#print(llm.generate("The capital of France is ", max_tokens=1028))

In [ ]:
llm

In [ ]:
shutil.copytree('/kaggle/input/mkvc-2-finalmix/kaggle/working/mk-vectordb-chroma-two', '/kaggle/working/mk-vectordb-chroma')

In [ ]:
%%time

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="firqaaa/indo-sentence-bert-base", model_kwargs={"device": "cuda"})

vectordb = Chroma(persist_directory='mk-vectordb-chroma',embedding_function=instructor_embeddings,)
#vectordb.persist()

In [ ]:
%%time
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
#retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25})

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True,
                                       verbose=False)

In [ ]:
qa_chain

In [ ]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
def llm_ans(query):
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    return ans

In [ ]:
#model

In [ ]:
dataset = pd.read_csv('/kaggle/input/mkri-qa-3/dataset.csv',sep=';')
dataset

In [ ]:
%%time
for x,y in zip(dataset.no,dataset.qa_comb):
    llm_time_start = time.time()
    print("Nomor: ",x)
    print(y)
    llm_ans(y)
    llm_time_end = time.time()
    print("Time: ",llm_time_end - llm_time_start)
    print("==============================================")

In [ ]:
end = time.time()
print(end - start)